In [36]:
import networkx as nx
import logging
import multiprocessing as mp
import pandas as pd
import sys
log = logging.getLogger("regulatory")
log.setLevel(logging.DEBUG)

http://www.regnetworkweb.org/source.jsp

In [113]:
g = nx.Graph()

In [114]:
def process_ENCODE_row(row):
    miRNA, gene = row.replace("\n", "").split("\t")[:2]
    if "M" not in miRNA: #miRNA regulate genes, first edge should be mirna
        return None
    else:
        return (miRNA, gene)
    
with open("human/ENCODE", "r") as file:
    pool = mp.Pool()
    w = pool.map_async(process_ENCODE_row, file.readlines(), error_callback=lambda err: log.error(err))
    pool.close()
    
pool.join()

g.add_edges_from([wi for wi in w.get() if wi is not None])

In [115]:
nodes = g.nodes()

def process_row(row):
        node = row.replace("\n", "").split("\t")
        if node[0] in nodes:
            return node[0], {"name":node[1].replace("hsa-miR", "hsa-mir")}
        else:
            return None
        
with open("human/human.node", "r") as file:
    pool = mp.Pool()
    w = pool.map_async(process_row, file.readlines(), error_callback=lambda err: log.error(err))
    pool.close()
    
pool.join()
g.add_nodes_from([wi for wi in w.get() if wi is not None])

In [116]:
df_adjiacency = pd.DataFrame(g.nodes(data=True), columns = ["id", "data"]).set_index("id")
def get_name(id):
    try:
        return df_adjiacency.loc[id,"data"]["name"]
    except:
        return None
df_adjiacency["name"] = list(map(get_name, df_adjiacency.index))

def get_name(id):
    try:
        return df_adjiacency.at[id,"name"]
    except:
        return None

In [117]:
df_adjiacency.head()

,data,name
id,,
MIMAT0000062,{'name': 'hsa-let-7a-5p'},hsa-let-7a-5p
10072,{'name': 'DPP3'},DPP3
10076,{'name': 'PTPRU'},PTPRU
10138,{'name': 'YAF2'},YAF2
10297,{'name': 'APC2'},APC2


In [123]:
for gene in list(map(get_name,g.neighbors("MIMAT0002882"))):
    print(gene)

PPIF
SERF2
MRVI1
CEBPB
PAPD7
GPRASP2
NR2C2AP
TCEANC2
VTI1A
E2F7
AP2B1
DDX6
E2F2
EGR1
HNRNPA3
FGFR1
PLA2G15
FLRT3
DPCD
CLIP3
ZBTB44
HUNK
HLF
HOXC10
KCNC1
MAF
ALDH6A1
BCL2L15
MYO6
NCAM1
NOVA1
PCDH7
PIGA
PLXNA2
CSNK1G1
C1orf123
PPP2R5E
SMEK1
ENAH
RAB22A
TSHZ3
GPR107
ZNF410
SCD
SFRP2
SPOCK1
STX3
VCP
TRIM26
MAPKAPK3
SLTM
SRCIN1
MLL2
ST8SIA2
CAMK2G
CCDC115
CCDC97
DLGAP2
CENPBD1
CDH1


In [119]:
nx.write_gpickle(g, "graph.pkl.gz")

In [120]:
df_adjiacency.reset_index().to_csv("graph_metadata.csv")